In [238]:
from collections import Counter
from itertools import combinations, chain

import networkx
from networkx import connected_components
import numpy as np
import pandas as pd

In [239]:
pd.set_option('display.max_columns', None)

In [240]:
def make_cds_ends_arr(gene):
    return ens_pc.loc[gene, ['cds_start', 'cds_end']].to_numpy()

def make_cds_ends_unique(cds_ends_arr):
    return np.sort(pd.unique(cds_ends_arr.ravel()))

def make_cds_ends_mtx(cds_ends_arr):
    return (make_cds_ends_unique(cds_ends_arr) == cds_ends_arr[..., None]).any(1).astype(int)

def make_exons_mtx(gene, cds_ends_unique):
    chrom = ens_pc.loc[gene].iloc[0, 0]
    gene_start = ens_pc.loc[gene].iloc[0, 4]
    gene_end = ens_pc.loc[gene].iloc[0, 5]
    
    trs = ens_exons[(ens_exons['chromosome'] == chrom) & (ens_exons['exon_start'].between(gene_start, gene_end) | ens_exons['exon_end'].between(gene_start, gene_end))]['ensembl_trs_id'].unique()
    trs = ens_other[ens_other['ensembl_trs_id'].isin(trs)]['ensembl_trs_id']
    x = (ens_exons[ens_exons['ensembl_trs_id'].isin(trs)].groupby('ensembl_trs_id')[['exon_start', 'exon_end']]
         .apply(lambda z: [np.arange(a, b+1) for a, b in zip(z['exon_start'], z['exon_end'])]).apply(np.hstack))

    return np.vstack([np.isin(cds_ends_unique, xi).astype(int) for xi in x]).sum(axis=0)

In [241]:
ens_annot = pd.read_csv('../data/interim/ensembl_annotation_with_dups_20220705.csv', 
                        dtype={'chromosome_or_scaffold': 'O', 
                               '5utr_start': 'Int64', 
                               '5utr_end': 'Int64', 
                               '3utr_start': 'Int64', 
                               '3utr_end': 'Int64', 
                               'cds_start': 'Int64', 
                               'cds_end': 'Int64'})

ens_exons = pd.read_csv('../data/processed/ensembl_exons_20220705.csv', 
                        dtype={'5utr_start': 'Int64', 
                               '5utr_end': 'Int64', 
                               '3utr_start': 'Int64', 
                               '3utr_end': 'Int64', 
                               'cds_start': 'Int64', 
                               'cds_end': 'Int64'})

In [242]:
ens_exons['chromosome'] = ens_exons['ensembl_gene_id'].map(ens_annot.drop_duplicates('ensembl_gene_id').set_index('ensembl_gene_id')['chromosome_or_scaffold'])

In [243]:
hek = pd.read_csv('../data/custom/va_hek293t_expression_all_trs.csv')

In [244]:
ens_annot['TPM'] = ens_annot['ensembl_trs_id'].map(hek.set_index('ensembl_trs_id')['TPM'])
ens_annot['TPM'] = ens_annot['TPM'].fillna(0)

In [250]:
ens_annot[ens_annot['TPM'] > 1].sort_values(['ensembl_gene_id', 'TPM'], ascending=False)

C:\Users\Dynev\AppData\Local\Temp\ipykernel_14332\576083815.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  ens_annot[ens_annot['TPM'] > 1].sort_values(['ensembl_gene_id', 'TPM'], ascending=False)[(~ens_annot2.duplicated(['ensembl_gene_id', '_protein_id'])) | (ens_annot2['_protein_id'].isna())]


IndexingError: Unalignable boolean Series provided as indexer (index of the boolean Series and of the indexed object do not match).

In [246]:
ens_annot.groupby('_protein_id', dropna=True)['TPM'].max()

_protein_id
A0A023T787             0.000000
A0A024QZ33             7.566501
A0A024QZ35             0.000000
A0A024QZ42            27.501784
A0A024QZ60, J3QKY8     1.145383
                        ...    
X6RLX0                 0.280590
X6RLY7                 0.000000
X6RM00                 0.045241
X6RM24                 3.543882
X6RM59                13.772463
Name: TPM, Length: 93924, dtype: float64

In [223]:
ens_annot2 = ens_annot[(~ens_annot.duplicated(['ensembl_gene_id', '_protein_id'])) | (ens_annot2['_protein_id'].isna())]

In [227]:
ens_pc = ens_annot2[(ens_annot2['trs_type'] == 'protein_coding')]
pc_size = ens_pc.groupby('ensembl_gene_id').size()
ens_pc = ens_pc[ens_pc['ensembl_gene_id'].isin(pc_size[pc_size.between(2, 10)].index)].set_index(['ensembl_gene_id', 'ensembl_trs_id'])
ens_other = ens_annot2[ens_annot2['ensembl_gene_id'].isin(ens_pc.index.get_level_values(0))].set_index(['ensembl_gene_id', 'ensembl_trs_id'])

In [225]:
ens_pc.index.get_level_values(0).nunique()

7821

In [198]:
ens_pc

chromosome_or_scaffold ensembl_gene_name  \
ensembl_gene_id ensembl_trs_id                                             
ENSG00000258992 ENST00000451548                      Y             TSPY1   
                ENST00000423647                      Y             TSPY1   
ENSG00000099715 ENST00000333703                      Y           PCDH11Y   
                ENST00000215473                      Y           PCDH11Y   
                ENST00000362095                      Y           PCDH11Y   
...                                                ...               ...   
ENSG00000143569 ENST00000613315                      1            UBAP2L   
                ENST00000428931                      1            UBAP2L   
ENSG00000143575 ENST00000457918                      1              HAX1   
                ENST00000483970                      1              HAX1   
                ENST00000328703                      1              HAX1   

                                      gene_type  strand  gene_start  \
ensembl_gene_id ensembl_trs_id                                        
ENSG00000258992 ENST00000451548  protein_coding       1     9466955   
                ENST00000423647  protein_coding       1     9466955   
ENSG00000099715 ENST00000333703  protein_coding       1     5000226   
                ENST00000215473  protein_coding       1     5000226   
                ENST00000362095  protein_coding       1     5000226   
...                                         ...     ...         ...   
ENSG00000143569 ENST00000613315  protein_coding       1   154220179   
                ENST00000428931  protein_coding       1   154220179   
ENSG00000143575 ENST00000457918  protein_coding       1   154272589   
                ENST00000483970  protein_coding       1   154272589   
                ENST00000328703  protein_coding       1   154272589   

                                  gene_end ensembl_trs_name        trs_type  \
ensembl_gene_id ensembl_trs_id                                                
ENSG00000258992 ENST00000451548    9469749        TSPY1-202  protein_coding   
                ENST00000423647    9469749        TSPY1-201  protein_coding   
ENSG00000099715 ENST00000333703    5742224      PCDH11Y-202  protein_coding   
                ENST00000215473    5742224      PCDH11Y-201  protein_coding   
                ENST00000362095    5742224      PCDH11Y-203  protein_coding   
...                                    ...              ...             ...   
ENSG00000143569 ENST00000613315  154271510       UBAP2L-220  protein_coding   
                ENST00000428931  154271510       UBAP2L-207  protein_coding   
ENSG00000143575 ENST00000457918  154275875         HAX1-204  protein_coding   
                ENST00000483970  154275875         HAX1-208  protein_coding   
                ENST00000328703  154275875         HAX1-201  protein_coding   

                                 is_ensembl_canonical trs_support_level  \
ensembl_gene_id ensembl_trs_id                                            
ENSG00000258992 ENST00000451548                  True              tsl1   
                ENST00000423647                 False              tsl1   
ENSG00000099715 ENST00000333703                 False              tsl1   
                ENST00000215473                 False              tsl1   
                ENST00000362095                 False              tsl1   
...                                               ...               ...   
ENSG00000143569 ENST00000613315                 False              tsl2   
                ENST00000428931                  True              tsl5   
ENSG00000143575 ENST00000457918                 False              tsl1   
                ENST00000483970                 False              tsl2   
                ENST00000328703                  True              tsl1   

                                 trs_length_bp  trs_start    trs_end  \
ensembl_gene_id ensembl_trs_id             

In [216]:
def main():
    results = {}
    genes = ens_pc.index.get_level_values(0).unique()

    for i, gene in enumerate(genes):
        cds_ends_arr = make_cds_ends_arr(gene)
        cds_ends_unique = make_cds_ends_unique(cds_ends_arr)
        cds_ends_mtx = make_cds_ends_mtx(cds_ends_arr)

        cds_ends_mtx_sub = cds_ends_mtx - cds_ends_mtx.sum(0)
        cds_ends_mtx_zeros = np.count_nonzero(cds_ends_mtx_sub==0, axis=1)
        possible_solutions = np.where(cds_ends_mtx_zeros == 1)[0]
        if not len(possible_solutions) >= 2:
            print(f'{i+1} / {len(genes)}         ', end='\r')
            continue

        exons_mtx = make_exons_mtx(gene, cds_ends_unique)

        solutions = []
        for indices in chain.from_iterable(combinations(possible_solutions, r) for r in range(2, len(possible_solutions)+1)):
            cds_sum_loc = cds_ends_mtx[indices, :].sum(axis=0)
            c = Counter(cds_sum_loc)
            if c[len(indices)] == 1 and c[1] == len(indices):
                cds_full_sub = np.where(cds_sum_loc != 0, cds_sum_loc - exons_mtx, 0)
                if (cds_full_sub == 0).all():
                    solutions.append(indices)

        if solutions:
            solutions_ids = []
            for j in solutions:
                solutions_ids.append(ens_pc.loc[gene].index[list(j)].to_list())
            results[gene] = solutions_ids
            print(f'{i+1} / {len(genes)}         ', end='\r')
        else:
            print(f'{i+1} / {len(genes)}         ', end='\r')
            continue
    
    return results

In [212]:
results = pd.Series(results).explode().explode().to_frame()

In [213]:
results2 = results.groupby(level=0).transform(lambda x: ', '.join(x)).drop_duplicates()

In [214]:
results2.to_csv('../reports/solutions_uniprot_pc_vs_gencode.txt', header=None, sep='\t')

In [235]:
ens_annot2 = ens_annot[ens_annot['in_gencode_basic']]
ens_annot2 = ens_annot2[(~ens_annot2.duplicated(['ensembl_gene_id', '_protein_id'])) | (ens_annot2['_protein_id'].isna())]

ens_pc = ens_annot2[(ens_annot2['trs_type'] == 'protein_coding')]
pc_size = ens_pc.groupby('ensembl_gene_id').size()
ens_pc = ens_pc[ens_pc['ensembl_gene_id'].isin(pc_size[pc_size.between(2, 10)].index)].set_index(['ensembl_gene_id', 'ensembl_trs_id'])

results = main()

results = pd.Series(results).explode().explode().to_frame()
results = results.groupby(level=0).transform(lambda x: ', '.join(x)).drop_duplicates()
results.to_csv('../reports/solutions_global_gencode_vs_gencode.txt', header=None, sep='\t')

In [237]:
results

,0
ENSG00000172288,"ENST00000361963, ENST00000306609"
ENSG00000172352,"ENST00000306882, ENST00000382407"
ENSG00000174015,"ENST00000310521, ENST00000378966"
ENSG00000133119,"ENST00000380071, ENST00000434425"
ENSG00000278558,"ENST00000612978, ENST00000613597"
...,...
ENSG00000137968,"ENST00000370859, ENST00000370855"
ENSG00000180667,"ENST00000315927, ENST00000367084"
ENSG00000132716,"ENST00000368073, ENST00000610139"
ENSG00000143297,"ENST00000361835, ENST00000368190, ENST00000368189"


In [253]:
for i in ['global', 'hek']:
    for j in ['all', 'gencode', 'uniprot']:
        for k in ['all', 'gencode']:
            if (j == 'all') & (k == 'gencode'):
                continue
            if (j == 'gencode') & (k == 'all'):
                continue
                
            if i == 'hek':
                ens_annot2 = ens_annot[ens_annot['TPM'] > 1]
                ens_annot2 = ens_annot2.sort_values(['ensembl_gene_id', 'TPM'], ascending=False)
            else:
                ens_annot2 = ens_annot.copy()
                
            ens_annot2 = ens_annot2[(~ens_annot2.duplicated(['ensembl_gene_id', '_protein_id'])) | (ens_annot2['_protein_id'].isna())]
            
            if j == 'all':
                ens_pc = ens_annot2[ens_annot2['trs_type'] == 'protein_coding']
            elif j == 'gencode':
                ens_pc = ens_annot2[(ens_annot2['trs_type'] == 'protein_coding') & (ens_annot2['in_gencode_basic'])]
            elif j == 'uniprot':
                ens_pc = ens_annot2[(ens_annot2['trs_type'] == 'protein_coding') & (~ens_annot2['uniprot_isoform_id'].isna())]
                
            pc_size = ens_pc.groupby('ensembl_gene_id').size()
            ens_pc = ens_pc[ens_pc['ensembl_gene_id'].isin(pc_size[pc_size.between(2, 10)].index)].set_index(['ensembl_gene_id', 'ensembl_trs_id'])
            
            if k == 'all':
                ens_other = ens_annot2.copy()
            else:
                ens_other = ens_annot2[ens_annot2['in_gencode_basic']]
                
            print(f'{i}: {j} vs {k} = {ens_pc.index.get_level_values(0).nunique()} genes vs {ens_other["ensembl_trs_id"].nunique()} trs.')
            
            results = main()

            results = pd.Series(results).explode().explode().to_frame()
            results = results.groupby(level=0).transform(lambda x: ', '.join(x)).drop_duplicates()
            results.to_csv(f'../reports/solution_{i}_{j}_vs_{k}.txt', header=None, sep='\t')

global: all vs all = 13312 genes vs 232030 trs.
global: gencode vs gencode = 11572 genes vs 103194 trs.
global: uniprot vs all = 7821 genes vs 232030 trs.
global: uniprot vs gencode = 7821 genes vs 103194 trs.
hek: all vs all = 7334 genes vs 54832 trs.
hek: gencode vs gencode = 6008 genes vs 25155 trs.
hek: uniprot vs all = 3995 genes vs 54832 trs.
hek: uniprot vs gencode = 3995 genes vs 25155 trs.


In [254]:
import glob

In [255]:
solutions = glob.glob('../reports/solution_*.txt')

In [280]:
for s in solutions:
    with open(s, 'r') as f:
        f = f.readlines()
        f = [i[:i.index('\t')] for i in f]
        with open(f"../reports/{s[s.index('_')+1:s.rindex('.')]}.txt", 'w') as g:
            g.writelines('\n'.join(f))

In [264]:
ens_annot[(ens_annot['trs_type'] == 'protein_coding') & (~ens_annot['uniprot_isoform_id'].isna())][ens_annot['ensembl_gene_id'] == 'ENSG00000162585']

C:\Users\Dynev\AppData\Local\Temp\ipykernel_14332\4129703627.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  ens_annot[(ens_annot['trs_type'] == 'protein_coding') & (~ens_annot['uniprot_isoform_id'].isna())][ens_annot['ensembl_gene_id'] == 'ENSG00000162585']


,ensembl_trs_id,chromosome_or_scaffold,ensembl_gene_id,ensembl_gene_name,gene_type,strand,gene_start,gene_end,ensembl_trs_name,trs_type,is_ensembl_canonical,trs_support_level,trs_length_bp,trs_start,trs_end,5utr_start,5utr_end,3utr_start,3utr_end,cds_start,cds_end,cds_length_bp,in_gencode_basic,appris_annotation,ensembl_protein_id,uniprot_base_id,uniprot_isoform_id,uniprot_trembl_id,ccds_id,protein_length_aa,_protein_id,TPM
78684,ENST00000400918,1,ENSG00000162585,FAAP20,protein_coding,-1,2184461,2212720,FAAP20-203,protein_coding,False,tsl2,811,2185175,2194751,2194750,2194751,2185175,2185470,2185471,2194749,513.0,False,NaN,ENSP00000383709,Q6NZ36,Q6NZ36-5,NaN,CCDS72687,170.0,Q6NZ36-5,4.764169
78688,ENST00000378543,1,ENSG00000162585,FAAP20,protein_coding,-1,2184461,2212720,FAAP20-201,protein_coding,False,tsl2,569,2189548,2194750,2194750,2194750,2189548,2189776,2189777,2194749,339.0,False,NaN,ENSP00000367804,Q6NZ36,Q6NZ36-6,NaN,CCDS57965,112.0,Q6NZ36-6,6.434441
78690,ENST00000378546,1,ENSG00000162585,FAAP20,protein_coding,-1,2184461,2212720,FAAP20-202,protein_coding,True,tsl1,727,2189548,2194772,2194750,2194772,2189548,2189708,2189709,2194749,543.0,True,principal1,ENSP00000367808,Q6NZ36,Q6NZ36-1,NaN,CCDS38,180.0,Q6NZ36-1,19.720081


In [261]:
ens_pc2 = ens_annot[(ens_annot['trs_type'] == 'protein_coding') & (~ens_annot['uniprot_isoform_id'].isna())]

In [262]:
ens_pc2

,ensembl_trs_id,chromosome_or_scaffold,ensembl_gene_id,ensembl_gene_name,gene_type,strand,gene_start,gene_end,ensembl_trs_name,trs_type,is_ensembl_canonical,trs_support_level,trs_length_bp,trs_start,trs_end,5utr_start,5utr_end,3utr_start,3utr_end,cds_start,cds_end,cds_length_bp,in_gencode_basic,appris_annotation,ensembl_protein_id,uniprot_base_id,uniprot_isoform_id,uniprot_trembl_id,ccds_id,protein_length_aa,_protein_id,TPM
46,ENST00000303728,Y,ENSG00000169789,PRY,protein_coding,1,22490397,22514637,PRY-201,protein_coding,True,tsl1,1238,22490397,22514637,22490397,22501564,22514071,22514637,22501565,22514070,444.0,True,principal1,ENSP00000302319,O14603,O14603-1,A0A384MTZ8,CCDS14799,147.0,O14603-1,0.000000
77,ENST00000651177,Y,ENSG00000114374,USP9Y,protein_coding,1,12537650,12860839,USP9Y-209,protein_coding,False,tslNA,9640,12662368,12860802,12662368,12709447,12859417,12860802,12709448,12859416,7668.0,True,principal1,ENSP00000498372,O00507,O00507-1,NaN,CCDS14781,2555.0,O00507-1,0.000000
79,ENST00000338981,Y,ENSG00000114374,USP9Y,protein_coding,1,12537650,12860839,USP9Y-201,protein_coding,True,tsl1,10036,12701231,12860839,12701231,12709447,12859417,12860839,12709448,12859416,7668.0,True,principal1,ENSP00000342812,O00507,O00507-1,NaN,CCDS14781,2555.0,O00507-1,0.000000
89,ENST00000303804,Y,ENSG00000169807,PRY2,protein_coding,-1,22071756,22096007,PRY2-201,protein_coding,True,tsl1,1238,22071756,22096007,22084840,22096007,22071756,22072322,22072323,22084839,444.0,True,principal1,ENSP00000303300,O14603,O14603-1,A0A384MTZ8,CCDS35482,147.0,O14603-1,0.000000
121,ENST00000361365,Y,ENSG00000198692,EIF1AY,protein_coding,1,20575776,20593154,EIF1AY-201,protein_coding,True,tsl1,1339,20575776,20593154,20575776,20575871,20592347,20593154,20575872,20592346,435.0,True,principal1,ENSP00000354722,O14602,O14602-1,NaN,CCDS14795,144.0,O14602-1,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246389,ENST00000428931,1,ENSG00000143569,UBAP2L,protein_coding,1,154220179,154271510,UBAP2L-207,protein_coding,True,tsl5,3877,154220955,154270847,154220955,154225123,154270296,154270847,154225124,154270295,3264.0,True,principal3,ENSP00000389445,Q14157,Q14157-2,NaN,CCDS1063,1087.0,Q14157-2,16.979130
246392,ENST00000361546,1,ENSG00000143569,UBAP2L,protein_coding,1,154220179,154271510,UBAP2L-203,protein_coding,False,tsl1,3864,154225082,154270853,154225082,154225123,154270296,154270853,154225124,154270295,3264.0,True,principal3,ENSP00000355343,Q14157,Q14157-2,NaN,CCDS1063,1087.0,Q14157-2,15.315751
246404,ENST00000457918,1,ENSG00000143575,HAX1,protein_coding,1,154272589,154275875,HAX1-204,protein_coding,False,tsl1,914,154272589,154275784,154272589,154272723,154275702,154275784,154272724,154275701,696.0,True,NaN,ENSP00000411448,O00165,O00165-5,A0A0S2Z565,CCDS44230,231.0,O00165-5,29.660585
246405,ENST00000483970,1,ENSG00000143575,HAX1,protein_coding,1,154272589,154275875,HAX1-208,protein_coding,False,tsl2,1151,154272609,154275873,154272609,154272723,154275702,154275873,154272724,154275701,864.0,True,NaN,ENSP00000435088,O00165,O00165-2,NaN,NaN,287.0,O00165-2,30.798806


In [267]:
with open('../reports\solution_global_all_vs_all.txt', 'r') as f:
    f = f.readlines()
    f = [i[:i.index('\t')] for i in f]

In [275]:
solutions[0][solutions[0].index('_')+1:solutions[0].rindex('.')]

'global_all_vs_all'

In [272]:
solutions[0].index('_')+2

21